### UC OSS Bedrock Agent Demo notebook 
### Shorter version to test existing agents and functions/tools.

In [3]:
import logging
import sys

from unitycatalog.ai.bedrock.toolkit import UCFunctionToolkit
from unitycatalog.ai.core.client import UnitycatalogFunctionClient
from unitycatalog.client import ApiClient, Configuration

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
import pprint

from unitycatalog.ai.bedrock.envs.bedrock_env_vars import BedrockEnvVars

bedrock_env = BedrockEnvVars.get_instance(load_from_file=True)
success = bedrock_env.load_from_file()

if success:
    logger.info(
        f"Bedrock Environment Variables:\n{pprint.pformat(bedrock_env.as_dict(), indent=2, width=100)}"
    )

In [21]:
UNITY_CATALOG_SERVER = "localhost:8080"
CATALOG = "AICatalog"
SCHEMA = "AISchema"

In [22]:
config = Configuration()
config.host = f"http://{UNITY_CATALOG_SERVER}/api/2.1/unity-catalog"

api_client = ApiClient(configuration=config)
uc_client = UnitycatalogFunctionClient(api_client=api_client)

In [ ]:
function_names = [
    f"{CATALOG}.{SCHEMA}.get_weather_in_celsius",
    f"{CATALOG}.{SCHEMA}.get_weather_in_fahrenheit",
]
uc_f_toolkit = UCFunctionToolkit(function_names=function_names, client=uc_client)
logger.info(uc_f_toolkit)

### Toolkit usage with single action group and multiple functions/tools

In [24]:
import uuid

# Create a session
session_respone = uc_f_toolkit.create_session(
    agent_id=bedrock_env.bedrock_agent_id,
    agent_alias_id=bedrock_env.bedrock_agent_alias_id,
    catalog_name=CATALOG,
    schema_name=SCHEMA,
)
# Generate unique session ID
session_id = str(uuid.uuid1())

In [ ]:
final_response = session_respone.invoke_agent(
    input_text="What is the weather in Centigrade and Fahrenheit for location 12345 and date of 2025-02-26?",
    enable_trace=True,
    session_id=session_id,
    uc_client=uc_f_toolkit.client,
)

In [ ]:
logger.info(f" Resonse from bedrock agent: {final_response.response_body}")